# FAANG Stock Min-Max (Part 1)

## Bloomberg SQL Interview Question

### Question

The Bloomberg terminal is the go-to resource for financial professionals, offering convenient access to a wide array of financial datasets. As a Data Analyst at Bloomberg, you have access to historical data on stock performance.

Currently, you're analyzing the highest and lowest open prices for each FAANG stock by month over the years.

For each FAANG stock, display the ticker symbol, the month and year ('Mon-YYYY') with the corresponding highest and lowest open prices (refer to the Example Output format). Ensure that the results are sorted by ticker symbol.

---

### stock_prices Schema:

| Column Name | Type     | Description                                                                 |
|-------------|----------|-----------------------------------------------------------------------------|
| date        | datetime | The specified date (mm/dd/yyyy) of the stock data.                          |
| ticker      | varchar  | The stock ticker symbol (e.g., AAPL) for the corresponding company.        |
| open        | decimal  | The opening price of the stock at the start of the trading day.             |
| high        | decimal  | The highest price reached by the stock during the trading day.              |
| low         | decimal  | The lowest price reached by the stock during the trading day.               |
| close       | decimal  | The closing price of the stock at the end of the trading day.               |

---

### Example Input for `stock_prices` Table:

| date                | ticker | open   | high   | low    | close  |
|---------------------|--------|--------|--------|--------|--------|
| 01/31/2023 00:00:00 | AAPL   | 142.28 | 142.70 | 144.34 | 144.29 |
| 02/28/2023 00:00:00 | AAPL   | 146.83 | 147.05 | 149.08 | 147.41 |
| 03/31/2023 00:00:00 | AAPL   | 161.91 | 162.44 | 165.00 | 164.90 |
| 04/30/2023 00:00:00 | AAPL   | 167.88 | 168.49 | 169.85 | 169.68 |
| 05/31/2023 00:00:00 | AAPL   | 176.76 | 177.33 | 179.35 | 177.25 |

---

### Example Output:

| ticker | highest_mth | highest_open | lowest_mth | lowest_open |
|--------|-------------|--------------|------------|-------------|
| AAPL   | May-2023    | 176.76       | Jan-2023   | 142.28      |

---

### Explanation

Apple Inc. (AAPL) achieved its highest opening price of **$176.76** in May 2023 and its lowest opening price of **$142.28** in January 2023.


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

# Create Spark session
spark = SparkSession.builder.master('local[1]').appName("StockPrices").getOrCreate()

# Define the schema with TimestampType for date
schema = StructType([
    StructField("date", TimestampType(), True),
    StructField("ticker", StringType(), True),
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True)
])

# Convert date strings to datetime objects in data
data = [
    (datetime.strptime("01/31/2023 00:00:00", "%m/%d/%Y %H:%M:%S"), "AAPL", 142.28, 142.70, 144.34, 144.29),
    (datetime.strptime("02/28/2023 00:00:00", "%m/%d/%Y %H:%M:%S"), "AAPL", 146.83, 147.05, 149.08, 147.41),
    (datetime.strptime("03/31/2023 00:00:00", "%m/%d/%Y %H:%M:%S"), "AAPL", 161.91, 162.44, 165.00, 164.90),
    (datetime.strptime("04/30/2023 00:00:00", "%m/%d/%Y %H:%M:%S"), "AAPL", 167.88, 168.49, 169.85, 169.68),
    (datetime.strptime("05/31/2023 00:00:00", "%m/%d/%Y %H:%M:%S"), "AAPL", 176.76, 177.33, 179.35, 177.25)
]

# Create the DataFrame
stock_prices_df = spark.createDataFrame(data, schema=schema)

# Show results and schema
stock_prices_df.show(truncate=False)
stock_prices_df.printSchema()


+-------------------+------+------+------+------+------+
|date               |ticker|open  |high  |low   |close |
+-------------------+------+------+------+------+------+
|2023-01-31 00:00:00|AAPL  |142.28|142.7 |144.34|144.29|
|2023-02-28 00:00:00|AAPL  |146.83|147.05|149.08|147.41|
|2023-03-31 00:00:00|AAPL  |161.91|162.44|165.0 |164.9 |
|2023-04-30 00:00:00|AAPL  |167.88|168.49|169.85|169.68|
|2023-05-31 00:00:00|AAPL  |176.76|177.33|179.35|177.25|
+-------------------+------+------+------+------+------+

root
 |-- date: timestamp (nullable = true)
 |-- ticker: string (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)



In [8]:
stock_prices_df.createOrReplaceTempView('stock_prices')

spark.sql(
'''
with 

cte as (SELECT 
  ticker,
  max(open) as highest_mth,
  min(open) as lowest_open
FROM stock_prices
GROUP BY ticker),

highest as (SELECT
  s.ticker,
  date_format(date, 'MMM-yyyy') as highest_mth,
  open as highest_open
FROM 
  stock_prices s JOIN cte c
  ON
  s.ticker=c.ticker AND open=highest_mth),
  
lowest as (SELECT
  s.ticker,
  date_format(date, 'MMM-yyyy') as lowest_mth,
  open as lowest_open
FROM 
  stock_prices s JOIN cte c
  ON
  s.ticker=c.ticker AND open=lowest_open)
  

select * 
from
  highest join lowest
  using(ticker)
ORDER BY 1
'''
).show()

+------+-----------+------------+----------+-----------+
|ticker|highest_mth|highest_open|lowest_mth|lowest_open|
+------+-----------+------------+----------+-----------+
|  AAPL|   May-2023|      176.76|  Jan-2023|     142.28|
+------+-----------+------------+----------+-----------+

